In [22]:
import GPy
import pandas as pd 
import numpy as np 
import os
import sys 
sys.path.append('../')
from dispersant_screener.definitions import FEATURES
DATADIR = '../data'

from dispersant_screener.pal import pal
from dispersant_screener.definitions import FEATURES
from dispersant_screener.utils import get_maxmin_samples, get_kmeans_samples 
from dispersant_screener.gp import build_model, build_coregionalized_model

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, RationalQuadratic, Matern, RationalQuadratic, DotProduct
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold

In [16]:
df_full_factorial_feat = pd.read_csv(os.path.join(DATADIR, 'new_features_full_random.csv'))[FEATURES].values
a2 = pd.read_csv(os.path.join(DATADIR, 'b1-b21_random_virial_large_new.csv'))['A2_normalized'].values
deltaGMax = pd.read_csv(os.path.join(DATADIR, 'b1-b21_random_virial_large_new.csv'))['A2_normalized'].values
gibbs = pd.read_csv(os.path.join(DATADIR, 'b1-b21_random_deltaG.csv'))['deltaGmin'].values  * (-1)
gibbs_max = pd.read_csv(os.path.join(DATADIR, 'b1-b21_random_virial_large_new.csv'))['deltaGmax'].values
force_max =  pd.read_csv(os.path.join(DATADIR, 'b1-b21_random_virial_large_fit2.csv'))['F_repel_max'].values
rg = pd.read_csv(os.path.join(DATADIR, 'rg_results.csv'))['Rg'].values
y = np.hstack([rg.reshape(-1,1), gibbs.reshape(-1,1), gibbs_max.reshape(-1,1)])
assert len(df_full_factorial_feat) == len(a2) == len(gibbs) == len(y)

In [42]:
vt = VarianceThreshold(0.2)
X = vt.fit_transform(df_full_factorial_feat)

feat_scaler = StandardScaler()
X = feat_scaler.fit_transform(X)

# label_scaler = StandardScaler()
# y = label_scaler.fit_transform(y)

X_train, y_train, greedy_indices = get_kmeans_samples(X, y, 20)


In [46]:
K = GPy.kern.RBF(X_train.shape[1])

In [47]:
icm = GPy.util.multioutput.ICM(input_dim=X_train.shape[1], num_outputs=3, kernel=K)

In [48]:
target_list = [y_train[:, i].reshape(-1, 1) for i in range(3)]

In [49]:
m = GPy.models.GPCoregionalizedRegression([X_train] * 3, target_list, kernel=icm)

In [50]:
m

gp.,value,constraints,priors
ICM.rbf.variance,1.0,+ve,
ICM.rbf.lengthscale,1.0,+ve,
ICM.B.W,"(3, 1)",,
ICM.B.kappa,"(3,)",+ve,
mixed_noise.Gaussian_noise_0.variance,1.0,+ve,
mixed_noise.Gaussian_noise_1.variance,1.0,+ve,
mixed_noise.Gaussian_noise_2.variance,1.0,+ve,


In [55]:
var = [i for i in m.parameter_names() if ('ICM' in i) and ('variance' in i)][0]

In [ ]:
'.*'

In [56]:
m['.*ICM.*.variance'].constrain_fixed(1.)

array([0])

In [57]:
m

gp.,value,constraints,priors
ICM.rbf.variance,1.0,+ve fixed,
ICM.rbf.lengthscale,1.0,+ve,
ICM.B.W,"(3, 1)",,
ICM.B.kappa,"(3,)",+ve,
mixed_noise.Gaussian_noise_0.variance,1.0,+ve,
mixed_noise.Gaussian_noise_1.variance,1.0,+ve,
mixed_noise.Gaussian_noise_2.variance,1.0,+ve,
